# 2950 Project Phase 2

Flavia Jiang (yj472), Rachel Wang (jw879)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.linear_model import LinearRegression, LogisticRegression
import duckdb

## Research Question

## Data Description

In [ ]:
1.What are the observations (rows) and the attributes (columns)?


2. Why was this dataset created?
This dataset was created to investigate and analyze various aspects of human dating behavior in the context of speed dating. 
Researchers were interested in understanding how individuals(males and females) make dating decisions, what attributes they 
consider important, and how different factors influence the outcomes of speed dating encounters.

3. Who funded the creation of the dataset?

The dataset was collected as part of academic research. 

4. What processes might have influenced what data was observed and recorded and what was not?

Participant Demographics: The age, gender, and demographic characteristics of the participants could influence the data collected. 
In this study, all subjects are from graduate and professional school of Columbia University.

Self-Selection of Participants: Participants in the speed dating experiment were volunteers, which means they self-selected to take part. 
This self-selection process may have introduced biases, as those who chose to participate might have different preferences or characteristics 
compared to the general population. This could impact the generalizability of the findings.

Experiment Design: The design of the speed dating experiment determined what data could be collected. The researchers structured the experiment, 
including the number of participants, the number of potential partners, and the available information about each partner. The experimental 
conditions may not fully represent real-world dating situations.

Survey Responses: The data collected was based on surveys and questionnaires filled out by participants. Data collection relied on participants' 
willingness to respond honestly and accurately, which could be influenced by social desirability bias or other factors.

5. What preprocessing was done, and how did the data come to be in the form that you are using?


6. If people are involved, were they aware of the data collection, and if so, what purpose did they expect the data to be used for?

Participants in the speed dating events would have been aware of the data collection process, as informed consent is a standard practice 
in research involving human subjects. They would have been informed about the purpose of the data collection, which is typically for academic 
research. Participants would have expected the data to be used to study dating behavior and potentially contribute to our understanding of human 
interactions and preferences.

7. Where can your raw source data be found, if applicable? Provide a link to the raw data

Link to the dataset and the documentation: http://www.stat.columbia.edu/~gelman/arm/examples/speed.dating/ 


## Data Cleaning

Firstly, load the data.

In [2]:
dating_df = pd.read_csv("speed_dating_data.csv", encoding="ISO-8859-1")
dating_df.shape

(8378, 195)

### Remove unnecessary columns
The data set is super wide, and there are so many variables. So we went through the variable codebook made by the creators of this data set and removed variables we currently think would be definitely unnecessary for our future analysis.

In [3]:
delete_list = ['idg', 
               'condtn', 
               'position', 
               'positin1', 
               'int_corr', 
               'undergra', 
               'mn_sat', 
               'zipcode', 
               'tuition', 
               'length', 
               'you_call', 
               'them_cal', 
               'numdat_2', 
               'num_in_3', 
               'go_out',
               'id', 
               'partner', 
               'int_corr', 
               'pf_o_att', 
               'pf_o_sin', 
               'pf_o_int', 
               'pf_o_fun', 
               'pf_o_amb', 
               'pf_o_sha', 
               'from', 
               'income', 
               'goal', 
               'date', 
               'exphappy', 
               'satis_2', 
               'date_3', 
               'numdat_3',
               'expnum']
    
if delete_list[0] in dating_df:
    dating_df = dating_df.drop(delete_list, axis = 1)
    
dating_df.shape

(8378, 163)

### Rename columns
Now, we still have many columns. Certainly, we won't use all of them, but we kept them there because they are potential predictors that might help train the model. And we don't want to restrict ourselves to a specific set of variables. 

Though we did not do any web scraping or merge data sets to get this data frame, one main effort we made was to interpret the meaning of each variable -- carefully reading through the 15-paged variable codebook. We noticed that some of the names of the given variable names were vague. So, we decided to change them so they conveyed more straightforward information about the variables. 

In [4]:
dating_df = dating_df.rename(columns = {"iid":"id",
                          "age_o":"partner_age",
                          "race_o":"partner_race", 
                          "round": "num_dates", 
                          "pid": "partner_id", 
                          "dec_o": "partner_decision", 
                          "match_es": "num_matches_estimated"})
dating_df.head()

,id,gender,wave,num_dates,order,partner_id,match,samerace,partner_age,partner_race,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,0,1,10,4,11.0,0,0,27.0,2.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,0,1,10,3,12.0,0,0,22.0,2.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,0,1,10,10,13.0,1,1,22.0,4.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,0,1,10,5,14.0,1,0,23.0,2.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,0,1,10,7,15.0,1,0,24.0,3.0,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


### Remove biased data points
As described in Data Description part, the researchers ran 21 speed dating sessions, or waves, in total. However, as they explained in their paper, they removed four sessions (waves 18-21) from the analysis "because they involved an experimental intervention where participants were asked to bring their favorite book. These four sessions were run specifically to study how decision weights and selectivity would be affected by an intervention designed to shift subjects’ attention away from superficial physical attributes. The inclusion of these four sessions does not alter the results reported below; they are omitted so that the only experimental difference across sessions is group size." Accordingly we also removed data for these four sessions. 

The researchers also said they removed another wave (#12) because they "imposed a maximum number of acceptances" on participants of this wave. We thought this restriction would affect participants' decisions, so we also removed this wave.

In [5]:
# F: filter (remove some waves, deal with NAs)
dating_df = dating_df[~dating_df['wave'].isin([12, 18, 19, 20, 21])]
dating_df.shape

(6412, 163)

The researchers also mentioned they removed two more sessions from their analysis because they thought their participants were too few. We thought this was not a big concern as of now. So we decided to keep them.

We had another concern, which the researchers did not address in their analysis. We noitced that for many of the survey questions, the researchers let participants of waves 6-9 rate the importance of six attributes separately on a scale of 1-10 and let participants of other waves distribute 100 points to these six attributes. Let's investigate this in the dataset with an example involving variables attr1_1, sinc1_1, intel1_1, fun1_1, amb1_1, shar1_1. 

In [6]:
temp = dating_df[(dating_df['wave'] > 5) & (dating_df['wave'] < 10)]
temp[['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].head()

,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1
1846,16.67,16.67,16.67,16.67,16.67,16.67
1847,16.67,16.67,16.67,16.67,16.67,16.67
1848,16.67,16.67,16.67,16.67,16.67,16.67
1849,16.67,16.67,16.67,16.67,16.67,16.67
1850,16.67,16.67,16.67,16.67,16.67,16.67


As we could infer from the values, the researchers converted the data for waves 6-9 so that for each set of variables for six attributes, they added up to 100 points. For example, if a participant of wave 6 rated these six attributes all as very important (that is, answered 10s to these six questions), it would be converted to (16.67, 16.67, 16.67, 16.67, 16.67, 16.67) so they summed to 100.

They did this so that they could still include waves 6-9 in the analysis. But this rude conversion could be problemetic. The simple logic is that the participant in the above example might think all six attributes were very important but not equally important. Then the adjusted data was not an accurate representation of the participant's beliefs.

Note that there were many sets of six variables like this because the researchers asked the participants to rate the importance of six attributes in different settings. And we were quite sure we would use at least one set of variable like this. So we decided to remove waves 6-9 from our analysis.

In [7]:
dating_df = dating_df[~dating_df['wave'].isin([6, 7, 8, 9])]
dating_df.shape

(4850, 163)

We know so far we had removed about half of the rows from the dataframe. But we still have 4000+ data points, which from a statistical standpoint are enough for regression and inference.

### Remove inaccurate data points

In [8]:
# R: verify 100 points (1_1, 4_1, 2_1, ...)

### Map coded categorical variables to their corresponding values

In [9]:
## F: categorical variable mapping: field, race, career

### Deal with missing values
We noticed for some variables there were many missing values due to how the experiment was designed and conducted. For each variable with missing values, we re-examined whether we still thought it would be a potential good predictor in our future modeling given the fact that including it would make the model less robust. We made decisions between keeping or removing the columns accordingly.

In [10]:
for col in dating_df:
    n = sum(pd.isna(dating_df[col]))
    if (n > 0):
        print([col, n])

['partner_id', 10]
['partner_age', 77]
['partner_race', 68]
['attr_o', 132]
['sinc_o', 182]
['intel_o', 188]
['fun_o', 215]
['amb_o', 435]
['shar_o', 661]
['like_o', 136]
['prob_o', 176]
['met_o', 231]
['age', 68]
['field', 58]
['field_cd', 77]
['race', 58]
['imprace', 74]
['imprelig', 74]
['career', 84]
['career_c', 133]
['sports', 74]
['tvsports', 74]
['exercise', 74]
['dining', 74]
['museums', 74]
['art', 74]
['hiking', 74]
['gaming', 74]
['clubbing', 74]
['reading', 74]
['tv', 74]
['theater', 74]
['movies', 74]
['concerts', 74]
['music', 74]
['shopping', 74]
['yoga', 74]
['attr1_1', 74]
['sinc1_1', 74]
['intel1_1', 74]
['fun1_1', 84]
['amb1_1', 94]
['shar1_1', 94]
['attr4_1', 1884]
['sinc4_1', 1884]
['intel4_1', 1884]
['fun4_1', 1884]
['amb4_1', 1884]
['shar4_1', 1884]
['attr2_1', 74]
['sinc2_1', 74]
['intel2_1', 74]
['fun2_1', 74]
['amb2_1', 84]
['shar2_1', 84]
['attr3_1', 100]
['sinc3_1', 100]
['fun3_1', 100]
['intel3_1', 100]
['amb3_1', 100]
['attr5_1', 1910]
['sinc5_1', 1910]
[

In [11]:
delete_list = []
for i in ['4_1', '4_2', '4_3', '5_1', '1_s', '3_s', '7_2', '2_2', '5_2', '1_3', '7_3', '2_3', '3_3', '5_3']:
    new = [var + i for var in ['attr', 'sinc', 'intel', 'fun', 'amb', 'shar']]
    delete_list = delete_list + new

for var in ['shar5_1', 'shar3_s', 'shar5_2', 'shar3_3', 'shar5_3']:
    delete_list.remove(var)

if delete_list[0] in dating_df:
    dating_df = dating_df.drop(delete_list, axis = 1)
    
dating_df.shape

(4850, 84)

Next, let's see how many rows would be left if all rows with any missing values were dropped.

In [12]:
dating_df.dropna().shape

(2578, 84)

A total of 2578 datapoints did not sound really bad, but we did not choose to do this because we were still deciding on the set of predictors. We should leave them there until we have final set of predictors based on the exploratory analysis below and the initial modeling phase. But from now on, we should be really carefully with NA values.

### Convert data types

In [13]:
# R: correct datatype (e.g. partner_id ...)

## Data Limitations


## Descriptive Analysis

In [14]:
# R: general: dimension
dating_df.shape

(4850, 84)

In [15]:
# R: one-variable: mean, sd, #observations
# for numerical variable: dating_df.describe()

In [16]:
# R: one-variable: plot distribution
# histogram for numeric variable (reference: kaggle example)
# pie chart for categorical variable (reference: discussion example)

In [17]:
# F: two variable
# change over Time1, during event, Time 2, Time 3
# correlation